# Lesson 02: Парсинг HTML. BeautifulSoup, MongoDB

1) Необходимо собрать информацию о вакансиях на вводимую должность (используем input или через аргументы) с сайта superjob.ru и hh.ru. Приложение должно анализировать несколько страниц сайта(также вводим через input или аргументы). Получившийся список должен содержать в себе минимум:

    *Наименование вакансии
    *Предлагаемую зарплату (отдельно мин. и и отдельно макс.)
    *Ссылку на саму вакансию        
    *Сайт откуда собрана вакансия

По своему желанию можно добавить еще работодателя и расположение. Данная структура должна быть одинаковая для вакансий с обоих сайтов. Общий результат можно вывести с помощью dataFrame через pandas.

In [21]:
import requests
from bs4 import BeautifulSoup as bs
import json
import pandas as pd

In [22]:
main_link = 'https://hh.kz/search/vacancy?area=40&st=searchVacancy&text=web+developer&page=0'
header = {'User-Agent': 'Chrome/78.0.3904.108 Safari/537.36'}

In [39]:
def hh_parse(main_link, header):
    """This function will parse to collect the jobs data from hh.kz website"""
    jobs_data = []
    links = []
    links.append(main_link)
    response = requests.get(main_link, headers=header).text
    html = bs(response, 'lxml')
    try: # to check how many pages needs to be parsed from website
        pagination = html.find_all('a', attrs={'data-qa': 'pager-page'})
        count = int(pagination[-1].text)
        for i in range(count):
            link = f'https://hh.kz/search/vacancy?area=40&st=searchVacancy&text=web+developer&page={i}'
            if link not in links:
                links.append(link)
    except:
        pass

    for link in links: # to parse each page to collect the data
        response = requests.get(link, headers=header).text
        html = bs(response, 'lxml')
        divs = html.find_all('div', attrs={'data-qa': 'vacancy-serp__vacancy'})
        for div in divs:
            try:
                vacancy_name = div.find('a', attrs={'data-qa': 'vacancy-serp__vacancy-title'}).text # to parse name of vacancy
                vacancy_salary = div.find('div', attrs={'data-qa': 'vacancy-serp__vacancy-compensation'}) # to parse how much salary is
                if vacancy_salary == None: # here will be divided to min and max level salary as well the currency
                    min_salary= 'None'
                    max_salary= 'None'
                    salary_currency = 'None'
                else:
                    vacancy_salary = div.find('div', attrs={'data-qa': 'vacancy-serp__vacancy-compensation'}).text.split(' ')
                    salary_currency = vacancy_salary[-1]
                    if vacancy_salary[0] == 'от':
                        min_salary = ''.join(vacancy_salary[1:-1])
                        max_salary = None
                    elif vacancy_salary[0] == 'до':
                        min_salary = ''.join(vacancy_salary[1:-1])
                        max_salary = None
                    else:
                        min_salary = vacancy_salary[0].split('-')[0]
                        max_salary = vacancy_salary[0].split('-')[1]

                vacancy_link = div.find('a', attrs={'data-qa': 'vacancy-serp__vacancy-title'})['href'] # to parse link of the vacancy
                company_name = div.find('a', attrs={'data-qa': 'vacancy-serp__vacancy-employer'}).text # to parse name of company
                # to collect all in 1 list
                jobs_data.append({
                    'vacancy_name': vacancy_name,
                    'company_name': company_name,
                    'min_salary': min_salary,
                    'max_salary': max_salary,
                    'salary_currency': salary_currency,
                    'vacancy_link': vacancy_link,
                })
                data = pd.DataFrame(jobs_data) # to convert from dict to DataFrame
                data.to_csv('data.csv', encoding='utf-16', index=False) #cp1251
            except:
                pass

In [40]:
hh_parse(main_link, header)

In [42]:
df = pd.read_csv('data.csv', encoding='utf-16')

In [43]:
df

,vacancy_name,company_name,min_salary,max_salary,salary_currency,vacancy_link
0,Backend Python Developer,ТОО AVIATA.KZ,750 000,NaN,KZT,https://hh.kz/vacancy/35395690?query=web%20dev...
1,Аsp.net developer,ТОО Norsec Delta Projects,None,None,None,https://hh.kz/vacancy/35179129?query=web%20dev...
2,Backend разработчик (Программист),ТОО Oktrening,250 000,500 000,KZT,https://hh.kz/vacancy/35364569?query=web%20dev...
3,Middle frontend-разработчик,ТОО AVIATA.KZ,150 000,750 000,KZT,https://hh.kz/vacancy/35284281?query=web%20dev...
4,Веб-программист,ТОО Effective Media Liaison,None,None,None,https://hh.kz/vacancy/35574626?query=web%20dev...
...,...,...,...,...,...,...
359,Программист 1С / 1С разработчик,ТОО «Страховая компания «Amanat»,300 000,800 000,KZT,https://hh.kz/vacancy/35178477?query=web%20dev...
360,Руководитель IT-отдела,ТОО CommunicationsTrust,250 000,NaN,KZT,https://hh.kz/vacancy/35182715?query=web%20dev...
361,Программист,ТОО Ареал-Транс,None,None,None,https://hh.kz/vacancy/35169932?query=web%20dev...
362,Главный-разработчик,ТОО Zumdex,None,None,None,https://hh.kz/vacancy/35313955?query=web%20dev...
